In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pickle
import re

#from the folders results_real load all csv files that start with dataFrame_results and put them in a dictionary with the name of the file as key
# Path: results_processing.ipynb
def load_results(path):
    results = {}
    for file in os.listdir(path):
        if file.startswith('dataFrame_results'):
            results[file[len("dataFrame_results"):-len('.csv')]] = pd.read_csv(os.path.join(path, file))
    return results

#load the results from the folder results_real
# Path: results_processing.ipynb
results = load_results('results_real')



In [2]:
mean_std_dictionary = {}
for metric, dataset in results.items():
    #put dataset.mean and dataset.std in a dataframe with the column names mean and std
    df = pd.DataFrame({'mean': dataset.mean(), 'std': dataset.std()})
    df = df.round(2)
    #put the dataframe in a dictionary with the name of the metric as key
    mean_std_dictionary[metric] = df

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_8132\1114752087.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = pd.DataFrame({'mean': dataset.mean(), 'std': dataset.std()})
C:\Users\Administrateur\AppData\Local\Temp\ipykernel_8132\1114752087.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = pd.DataFrame({'mean': dataset.mean(), 'std': dataset.std()})
C:\Users\Administrateur\AppData\Local\Temp\ipykernel_8132\1114752087.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a f

In [3]:
# Combine all the dataframes into one dataframe
df = pd.concat(mean_std_dictionary, axis=0)

# Unstack the dataframe to have models as columns and (metric, mean) and (metric, std) as multi-level indexes
unstacked_df = df.unstack()

# Transpose the dataframe to have models as columns and metrics as rows
unique_df = unstacked_df
unique_df = unique_df.swaplevel(0, 1, axis=1).sort_index(axis=0)

#reored the column putting mean and std besides
unique_df = unique_df.reindex(['mean', 'std'], axis=1, level=1)

unique_df = unique_df.reorder_levels([0, 1], axis=1)
unique_df = unique_df.sort_index(axis=1)

new_order = ['Precision', 'Recall', 'F1', 'PRAUC', 'PRAUC_C', 'Pk50', 'Pk100', 'Pk200', 'Pk500', 'Pk1000', 'Pk2000']
unique_df = unique_df.reindex(index=new_order)

#save to csv
unique_df.to_csv('publish/real_results_10d.csv', index=True, header=True)


# Regressor scores

In [4]:
def load_results(path):
    results = {}
    for file in os.listdir(path):
        if file.startswith('regressor_scores_'):
            results[file[len("regressor_scores_"):-len('.csv')]] = pd.read_csv(os.path.join(path, file))
    return results

#load the results from the folder results_real
# Path: results_processing.ipynb
results = load_results('results_real')

In [5]:
df = pd.concat(results.values(), keys=results.keys())

In [6]:
grouped = df.groupby(level=1)

result = grouped.agg(
    X_TERMINAL_mean=('X_TERMINAL', np.mean),
    X_TERMINAL_std=('X_TERMINAL', np.std),
    Y_TERMINAL_mean=('Y_TERMINAL', np.mean),
    Y_TERMINAL_std=('Y_TERMINAL', np.std),
    TX_AMOUNT_mean=('TX_AMOUNT', np.mean),
    TX_AMOUNT_std=('TX_AMOUNT', np.std),
).reset_index(drop=True)


In [7]:
result.round(2).to_csv('publish/real_results_10d_regressors.csv', index=True, header=True)

In [8]:
from PIL import Image

# for all png in the folder 
for file in os.listdir('results_real'):
    if file.endswith('.png'):
        # open the image
        img = Image.open(os.path.join('results_real', file))
        # convert to grayscale
        img = img.convert("L")
        # save the image
        #make dir publish/plots_synthetic
        if not os.path.exists('publish/plots_real'):
            os.makedirs('publish/plots_real')
        img.save(os.path.join('publish/plots_real', file[:-4] + '.eps'), "EPS")

# TRAPZ

In [15]:
names = ['Baseline','Baseline_balanced', 'SMOTE','Random', 'KMeansSMOTE', 'CTGAN','TIMEGAN', 'ADVO']

def load_results(path):
    results = {}
    for file in os.listdir(path):
        if file.startswith('trapz_'):
            dataframe = pd.read_csv(os.path.join(path, file))
            dataframe.index = names[2:]
            results[file[len("trapz_"):-len('.csv')]] = dataframe.round(4)
    return results

#load the results from the folder results_real
# Path: results_processing.ipynb
results = load_results('results_synthetic')

In [16]:
results

{'6':              X_TERMINAL  Y_TERMINAL  TX_AMOUNT
 SMOTE            0.1120      0.1162     0.1345
 Random           0.1052      0.1077     0.0586
 KMeansSMOTE      0.1039      0.1074     0.0579
 CTGAN            0.0607      0.0768     0.1066
 TIMEGAN          0.1287      0.1420     0.1399
 ADVO             0.1677      0.1713     0.0785,
 '21':              X_TERMINAL  Y_TERMINAL  TX_AMOUNT
 SMOTE            0.1121      0.1082     0.1301
 Random           0.1084      0.1031     0.0613
 KMeansSMOTE      0.1079      0.1029     0.0617
 CTGAN            0.1104      0.0994     0.1030
 TIMEGAN          0.1539      0.1599     0.1361
 ADVO             0.1654      0.1703     0.0980,
 '15':              X_TERMINAL  Y_TERMINAL  TX_AMOUNT
 SMOTE            0.1041      0.1092     0.1343
 Random           0.1026      0.1027     0.0585
 KMeansSMOTE      0.1009      0.1027     0.0593
 CTGAN            0.0856      0.0927     0.0807
 TIMEGAN          0.1274      0.1481     0.1325
 ADVO             0.1

In [20]:
import pandas as pd

# assuming 'results' is your dictionary of dataframes
df_list = [df for df in results.values()]

# concat all dataframes along the axis=0 (row-wise)
combined_df = pd.concat(df_list)

# calculate the mean for each method
mean_df = combined_df.groupby(combined_df.index).mean()
cols = []
for col in mean_df.columns:
    cols.append(f"{col}_MEAN")
mean_df.columns = cols
std_df = combined_df.groupby(combined_df.index).std()
cols = []
for col in std_df.columns:
    cols.append(f"{col}_STD")
std_df.columns = cols

concatenation = pd.concat([mean_df, std_df], axis=1)
reordering_cols = ["X_TERMINAL_MEAN", "X_TERMINAL_STD", "Y_TERMINAL_MEAN", "Y_TERMINAL_STD", "TX_AMOUNT_MEAN", "TX_AMOUNT_STD"]
concatenation = concatenation[reordering_cols]

concatenation.round(4).to_csv('trapzs.csv', index=True, header=True)



In [22]:
concatenation.round(4)

,X_TERMINAL_MEAN,X_TERMINAL_STD,Y_TERMINAL_MEAN,Y_TERMINAL_STD,TX_AMOUNT_MEAN,TX_AMOUNT_STD
ADVO,0.1673,0.0012,0.1712,0.0013,0.0841,0.0079
CTGAN,0.0960,0.0168,0.0990,0.0112,0.0748,0.0324
KMeansSMOTE,0.1063,0.0060,0.1033,0.0066,0.0553,0.0056
Random,0.1074,0.0059,0.1033,0.0067,0.0555,0.0055
SMOTE,0.1115,0.0069,0.1099,0.0071,0.1327,0.0049
TIMEGAN,0.1413,0.0117,0.1503,0.0076,0.1341,0.0064
